In [2]:
print("Om Namah Shivay")

Om Namah Shivay


In [3]:
import pandas as pd
import spacy
from rapidfuzz import process, fuzz


In [4]:

# Load medical NER model
nlp = spacy.load("en_ner_bc5cdr_md")


c:\Users\TanayShah\scispacy_env\lib\site-packages\spacy\language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [5]:

# Read and process symptom vocabulary
df = pd.read_csv("medical_knowledge.csv")
symptom_vocab = {s.strip().lower() for symptoms in df["Symptoms"] for s in symptoms.split(",")}

# Print to verify
print("Symptom Vocabulary:", sorted(symptom_vocab))

synonym_map = {
    "feeling nauseous": "nausea",
    "nauseous": "nausea",
    "throwing up": "vomiting",
    "tired": "fatigue",
    "weak": "weakness",
    "lost weight": "weight loss",
    "sore throat": "sore throat",
    "throat": "sore throat",
    "sensitive to light": "light sensitivity",
    "blurry vision": "blurred vision",
    "vision feels blurry": "blurred vision",
    "dehydrated": "dehydration",
    "pale stool": "pale stool",
    "pale stools": "pale stool",
    "swollen legs": "leg swelling",
    "swollen": "swelling",
    "sweats": "sweating"
}



Symptom Vocabulary: ['abdominal cramps', 'abdominal pain', 'aching', 'acid regurgitation', 'anal itching', 'anxiety', 'aura', 'blackheads', 'bleeding', 'bleeding tendency', 'blisters', 'bloating', 'blurred vision', 'brief spinning sensations', 'burning', 'chest pain', 'chest tightness', 'chills', 'cloudy urine', 'cold intolerance', 'confusion', 'congestion', 'constipation', 'cough', 'crepitus', 'dark urine', 'dehydration', 'diarrhea', 'diarrhea or constipation', 'dizziness', 'dry skin', 'elevated liver enzymes', 'fatigue', 'fever', 'frequent urge', 'frequent urination', 'headache', 'heartburn', 'heat intolerance', 'heaviness', 'hemoptysis', 'hives', 'hoarseness', 'honey-colored crusts', 'hunger', 'imbalance', 'increased thirst', 'itching', 'itchy eyes', 'itchy rash', 'jaundice', 'joint pain', 'leg swelling', 'light sensitivity', 'lower abdominal pain', 'lymphadenopathy', 'nail pitting', 'nausea', 'neck pain', 'night sweats', 'nighttime symptoms', 'nodules', 'nosebleeds', 'numbness', 'o

In [6]:

# Fuzzy matcher
def match_symptoms(extracted, vocab):
    matches = []
    for symptom in extracted:
        match, score, _ = process.extractOne(symptom.lower(), vocab, scorer=fuzz.token_sort_ratio)
        print(f"Matched '{symptom}' to '{match}' (score: {score})")
        if score > 80:
            matches.append(match)
    return matches



In [7]:

def extract_symptoms(text):
    doc = nlp(text)

    # Step 1: NER-based
    ner_entities = [ent.text.lower() for ent in doc.ents if ent.label_ == "DISEASE"]
    print("NER Extracted:", ner_entities)

    # Step 2: Manual synonym-based scan of input
    synonym_hits = []
    for phrase, canonical in synonym_map.items():
        if phrase in text.lower():
            synonym_hits.append(canonical)

    print("Synonym Matches:", synonym_hits)

    # Step 3: Keyword-based match from vocab
    keyword_matches = []
    for vocab_symptom in symptom_vocab:
        if vocab_symptom in text.lower():
            keyword_matches.append(vocab_symptom)
    print("Keyword Matches:", keyword_matches)

    # Combine all sources
    combined = list(set(ner_entities + synonym_hits + keyword_matches))

    # Step 4: Fuzzy match to canonical vocab
    final_matches = match_symptoms(combined, symptom_vocab)

    return list(set(final_matches))

In [8]:
synonym_map.update({
    "dizzy": "dizziness",
    "lightheaded": "dizziness",
    "lost some weight": "weight loss",
    "weight lost": "weight loss",
    "itchy": "itching",
    "itchy skin": "itching",
    "dehydrated": "dehydration",
    "pale stool": "pale stool",
    "pale stools": "pale stool",
    "urinate painfully": "painful urination"
})


In [9]:
#Testing
test_inputs = [
    "I've had a terrible headache and chills since last night.",
    "Feeling nauseous and dizzy with occasional vomiting.",
    "My throat is sore and I'm constantly sneezing with a runny nose.",
    "I feel extremely tired, weak, and have lost some weight recently.",
    "My skin is itchy with red rash and some small blisters forming.",
    "Experiencing chest pain and shortness of breath, especially when walking.",
    "I've been having diarrhea and abdominal cramps for two days.",
    "I have a weird burning sensation when I pee and some lower abdominal pain.",
    "Lately I’ve been really sensitive to light and my vision feels blurry.",
    "There’s a fever, night sweats, and I’m coughing up sputum.",
    "My legs feel swollen and I noticed some visible twisted veins.",
    "I’m vomiting, feeling dehydrated, and my stool looks pale."
]

# 2. Loop and test
for i, sentence in enumerate(test_inputs, 1):
    print(f"\nTest Case {i}: {sentence}")
    extracted = extract_symptoms(sentence)
    print(f"Extracted Symptoms: {extracted}")



Test Case 1: I've had a terrible headache and chills since last night.
NER Extracted: ['headache', 'chills']
Synonym Matches: []
Keyword Matches: ['chills', 'headache']
Matched 'headache' to 'headache' (score: 100.0)
Matched 'chills' to 'chills' (score: 100.0)
Extracted Symptoms: ['headache', 'chills']

Test Case 2: Feeling nauseous and dizzy with occasional vomiting.
NER Extracted: ['vomiting']
Synonym Matches: ['nausea', 'nausea', 'dizziness']
Keyword Matches: ['vomiting']
Matched 'vomiting' to 'vomiting' (score: 100.0)
Matched 'dizziness' to 'dizziness' (score: 100.0)
Matched 'nausea' to 'nausea' (score: 100.0)
Extracted Symptoms: ['vomiting', 'dizziness', 'nausea']

Test Case 3: My throat is sore and I'm constantly sneezing with a runny nose.
NER Extracted: ['throat', 'sore']
Synonym Matches: ['sore throat']
Keyword Matches: ['runny nose', 'sneezing']
Matched 'sore throat' to 'sore throat' (score: 100.0)
Matched 'sneezing' to 'sneezing' (score: 100.0)
Matched 'throat' to 'sore thr

Disease Prediction

In [10]:
# Convert disease symptom lists to sets
df["SymptomSet"] = df["Symptoms"].apply(lambda x: {s.strip().lower() for s in x.split(",")})

def predict_disease(user_symptoms, top_k=3):
    """Predict disease based on overlap with known symptoms."""
    predictions = []

    for _, row in df.iterrows():
        known_symptoms = row["SymptomSet"]
        match_count = len(set(user_symptoms) & known_symptoms)
        score = match_count / len(known_symptoms)  # Jaccard-like
        if match_count > 0:
            predictions.append((row["Disease"], score, match_count, known_symptoms))

    # Sort by score and match count
    predictions.sort(key=lambda x: (-x[1], -x[2]))

    # Return top K predictions
    top_predictions = predictions[:top_k]
    print("\n🩺 Top Disease Predictions:")
    for disease, score, count, symptoms in top_predictions:
        print(f"- {disease} (match: {count}, score: {score:.2f}, needed: {symptoms})")

    return [d[0] for d in top_predictions]

In [11]:
# Simulate input
text = "I've had a terrible headache and chills since last night."
extracted = extract_symptoms(text)  # from earlier step

# Predict diseases
predicted = predict_disease(extracted)


NER Extracted: ['headache', 'chills']
Synonym Matches: []
Keyword Matches: ['chills', 'headache']
Matched 'headache' to 'headache' (score: 100.0)
Matched 'chills' to 'chills' (score: 100.0)

🩺 Top Disease Predictions:
- Malaria (match: 2, score: 0.40, needed: {'headache', 'fever', 'sweating', 'chills', 'fatigue'})
- Hypoglycemia (match: 1, score: 0.20, needed: {'headache', 'trembling', 'sweating', 'hunger', 'confusion'})
- Hypertension (match: 1, score: 0.20, needed: {'headache', 'nosebleeds', 'chest pain', 'dizziness', 'blurred vision'})


In [12]:
def create_doc(row):
    return f"""Disease: {row['Disease']}
Symptoms: {row['Symptoms']}
Description: {row['Description']}
Treatment: {row['Treatment']}
Risk Factors: {row['Risk_Factors']}"""

docs = df.apply(create_doc, axis=1).tolist()

Retrieval

In [13]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = model.encode(docs)


c:\Users\TanayShah\scispacy_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\TanayShah\scispacy_env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\TanayShah\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to 

In [14]:
import faiss

dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)  # L2 distance
index.add(np.array(doc_embeddings))

# Optional: keep track of which index maps to which disease
id_to_disease = {i: df.iloc[i]["Disease"] for i in range(len(df))}


In [15]:
query = ", ".join(["headache", "chills"])
query_vector = model.encode([query])

top_k = 5
D, I = index.search(np.array(query_vector), top_k)

print("Top Matches:")
for i, idx in enumerate(I[0]):
    print(f"{i+1}. {id_to_disease[idx]} (score: {D[0][i]:.2f})")


Top Matches:
1. Migraine (score: 1.21)
2. Common Cold (score: 1.35)
3. Paralysis (brain hemorrhage) (score: 1.35)
4. Dengue (score: 1.38)
5. Malaria (score: 1.38)


Diagnosis

In [16]:
from langchain.docstore.document import Document

documents = [
    Document(page_content=docs[i], metadata={"source": df.iloc[i]["Disease"]})
    for i in range(len(df))
]
